**INSTALLING MPI LIBRARY**

In [1]:
!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.8 MB/s eta 0:00:00


**Creating mpi_program file**

In [2]:
%%writefile mpi_program.py
from mpi4py import MPI
import sys

comm = MPI.COMM_WORLD

rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    for i in range(1, size):
        message = comm.recv(source=i)
        print(f"Received from process {i}: {message}")
else:
  comm.send(f"Hello from process {rank}", dest=0)

Writing mpi_program.py


**installing mpich**

In [3]:
!apt-get install -y mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 2 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich-dev amd64 4.0-3 [7,375 kB]
Fetched 14.2 MB in 2

**Modfying the code to get the desired output**

In [4]:
%%writefile mpi_program.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    print("Master process receiving results...\n")
    for i in range(1, size):
        message = comm.recv(source=i)
        print(f"Received from process {message['rank']}:")
        print(f"  Assigned Task: Data Chunk {message['task']}")
        print(f"  Computed Sum: {message['result']}\n")
else:
    task_number = rank
    numbers = list(range(1, rank * 10 + 1))
    computed_sum = sum(numbers)

    message = {
        "rank": rank,
        "task": task_number,
        "result": computed_sum
    }

    comm.send(message, dest=0)


Overwriting mpi_program.py


**Running the code with mpiexec**

In [5]:
!mpiexec -n 4 python mpi_program.py


--------------------------------------------------------------------------
mpiexec has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

We strongly suggest that you run mpiexec as a non-root user.

You can override this protection by adding the --allow-run-as-root option
to the cmd line or by setting two environment variables in the following way:
the variable OMPI_ALLOW_RUN_AS_ROOT=1 to indicate the desire to override this
protection, and OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 to confirm the choice and
add one more layer of certainty that you want to do so.
We reiterate our advice against doing so - please proceed at your own risk.
--------------------------------------------------------------------------


**Enables the notebook to run as root**

In [6]:
import os
os.environ["OMPI_ALLOW_RUN_AS_ROOT"] = "1"
os.environ["OMPI_ALLOW_RUN_AS_ROOT_CONFIRM"] = "1"

**Running the code again with mpiexec**

In [8]:
!mpiexec --oversubscribe -n 4 python mpi_program.py


Master process receiving results...

Received from process 1:
  Assigned Task: Data Chunk 1
  Computed Sum: 55

Received from process 2:
  Assigned Task: Data Chunk 2
  Computed Sum: 210

Received from process 3:
  Assigned Task: Data Chunk 3
  Computed Sum: 465



**Reflection:**

1. **Why is message passing required in distributed systems?**
Message passing is required because distributed systems are made up of multiple computers or processes that do not share the same memory. Since they are separate, they cannot directly access each other’s data. Instead, they must communicate by sending messages back and forth. This allows them to share information, coordinate tasks, and work together to complete a larger job.
2. **What happens if one process fails?**
If one process fails, the system may lose part of its work. Other processes might stop waiting for a message that never arrives, which can cause delays or errors. In some systems, there are backup processes or error-handling methods to keep everything running smoothly. But in simpler systems, one failed process can affect the whole program.
3. **How does this model differ from shared-memory programming?**
In message passing, processes communicate by sending messages to each other because they do not share memory. In shared-memory programming, multiple processes or threads use the same memory space and can directly read or write the same data. Shared-memory systems are usually easier to program, but they can have problems like data conflicts. Message passing avoids direct memory conflicts but requires more careful communication between processes.